In [14]:
import numpy as np
import pandas as pd
import requests
import xlsxwriter
import math


In [ ]:
stocks = pd.read_csv('../Algorithmic-Python-Trading/static/sp-500-stocks/sp_500_stocks.csv')
stocks

Acquiring an API Token

In [16]:
from secret.secret import IEX_CLOUD_API_TOKEN

In [23]:

# symbol = 'AAPL'
# api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote/?token={IEX_CLOUD_API_TOKEN}'
api_url = f'https://api.iex.cloud/v1/data/core/quote/aapl?token={IEX_CLOUD_API_TOKEN}'
api_url
data = requests.get(api_url).json()
data[0]['high']

In [ ]:
symbol = data[0]['symbol']
price = data[0]['delayedPrice']
market_cap = data[0]['marketCap']
print(symbol)
print(price)
print(market_cap)

In [ ]:
my_columns = [ 'Ticker', 'Stock Price', 'Market Capitalization', 'Number of Shares to Buy']
final_dataframe = pd.DataFrame(columns = my_columns)
final_dataframe

In [ ]:
# final_dataframe.append(
#     pd.Series(
#         [
#             symbol,
#             price,
#             market_cap,
#             'N/A'
#         ],
#         index = my_columns
#     ),
#     ignore_index = True
# )
new_row = pd.Series([symbol, price, market_cap, 'N/A'], index=my_columns)
final_dataframe = pd.concat([final_dataframe, pd.DataFrame([new_row])], ignore_index=True)
final_dataframe

In [ ]:
final_dataframe = pd.DataFrame(columns=my_columns)
for stock in stocks['Ticker']:
    api_url = f'https://api.iex.cloud/v1/data/core/quote/aapl?token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(api_url).json()
    new_row = pd.Series([stock, data[0]['latestPrice'], market_cap, 'N/A'], index=my_columns)
    final_dataframe = pd.concat([final_dataframe, pd.DataFrame([new_row])], ignore_index=True)
final_dataframe

In [29]:
def chunks(lst, n):
    """Yield successive n-sized from list."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

In [ ]:
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))

final_dataframe = pd.DataFrame(columns= my_columns)
for symbol_string in symbol_strings:

    batch_api_call_url = f'https://api.iex.cloud/v1/data/core/quote/{symbol_string}?token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()

    for symbol in symbol_string.split(','):
        # Find the element with the matching symbol
        matching_symbol_data = next(item for item in data if item["symbol"] == symbol)
        # Extract the latestPrice
        latest_price = matching_symbol_data["latestPrice"]
        marketCap = matching_symbol_data["marketCap"]

        new_row = pd.Series([symbol, latest_price, marketCap, 'N/A'], index=my_columns)
        final_dataframe = pd.concat([final_dataframe, pd.DataFrame([new_row])], ignore_index=True)

final_dataframe


In [32]:
portfolio_size = input('Enter the value of your portfolio: ')

try: 
    val = float(portfolio_size)
except ValueError:
    print("That's not a valid number! Please try again:")
    portfolio_size = input('Enter the value of your portfolio: ')
    val = float(portfolio_size)

In [ ]:
print("Previous value: ", val)
position_size = val/len(final_dataframe.index)
for i in range(0, len(final_dataframe.index)):
    final_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / final_dataframe['Stock Price'][i])
final_dataframe

In [34]:
writer = pd.ExcelWriter('Recommended-trades.xlsx', engine = 'xlsxwriter')
final_dataframe.to_excel(writer, 'Recommended Trades', index = False)

In [35]:
background_color = '#0a0a23'
font_color = "#ffffff"

string_format = writer.book.add_format(
    {
        "font_color" : font_color,
        "bg_color" : background_color, 
        "border" : 1
    }
)

dollar_format = writer.book.add_format(
    {
        "num_format" : '$0.00',
        "font_color" : font_color,
        "bg_color" : background_color, 
        "border" : 1
    }
)

integer_format = writer.book.add_format(
    {
        "num_format" : '0',
        "font_color" : font_color,
        "bg_color" : background_color, 
        "border" : 1
    }
)

In [ ]:
# writer.sheets['Recommended Trades'].set_column('A:A', 18, string_format)
# writer.sheets['Recommended Trades'].set_column('B:B', 18, string_format)
# writer.sheets['Recommended Trades'].set_column('C:C', 18, string_format)
# writer.sheets['Recommended Trades'].set_column('D:D', 18, string_format)
# writer.close()

writer.sheets['Recommended Trades'].write('A1', 'Tricker', string_format)
writer.sheets['Recommended Trades'].write('B1', 'Stock Price', dollar_format)
writer.sheets['Recommended Trades'].write('C1', 'Market Capitalization', dollar_format)
writer.sheets['Recommended Trades'].write('D1', 'Number of Shares to Buy', integer_format)

In [37]:
column_formats = {
    'A': ['Ticker', string_format],
    'B': ['Stock Price', dollar_format],
    'C': ['Market Capitalization', dollar_format],
    'D': ['Number of Shares to Buy', integer_format]
}
for column in column_formats.keys():
    writer.sheets['Recommended Trades'].set_column(f'{column}:{column}', 20, column_formats[column][1])
    writer.sheets['Recommended Trades'].write(f'{column}1', column_formats[column][0], string_format)

writer.close()